In [1]:
import pickle
import cPickle
import sklearn
import numpy as np
import lasagne
import skimage.transform
import os
import theano
from lasagne.utils import floatX

/home/ubuntu/anaconda2/envs/pydata/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
Using gpu device 0: GRID K520 (CNMeM is disabled, CuDNN 3007)


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

### Load dataset

In [3]:
f = open('image_dataset.pkl', 'rb')
dataset = cPickle.load(f)
f.close()

In [4]:
len(dataset)

481810

### Get GoogLeNet

In [ ]:
!wget -N https://s3.amazonaws.com/lasagne/recipes/pretrained/imagenet/blvc_googlenet.pkl
!wget -N https://raw.githubusercontent.com/Lasagne/Recipes/master/modelzoo/googlenet.py
import googlenet
cnn_layers = googlenet.build_model()
cnn_input_var = cnn_layers['input'].input_var
cnn_feature_layer = cnn_layers['loss3/classifier']
cnn_output_layer = cnn_layers['prob']

get_cnn_features = theano.function([cnn_input_var], lasagne.layers.get_output(cnn_feature_layer))

In [ ]:
model_param_values = pickle.load(open('blvc_googlenet.pkl'))['param values']
lasagne.layers.set_all_param_values(cnn_output_layer, model_param_values)

In [8]:
MEAN_VALUES = np.array([104, 117, 123]).reshape((3,1,1))

def prep_image(im):
    if len(im.shape) == 2:
        im = im[:, :, np.newaxis]
        im = np.repeat(im, 3, axis=2)
    # Resize so smallest dim = 224, preserving aspect ratio
    h, w, _ = im.shape
    if h < w:
        im = skimage.transform.resize(im, (224, w*224/h), preserve_range=True)
    else:
        im = skimage.transform.resize(im, (h*224/w, 224), preserve_range=True)

    # Central crop to 224x224
    h, w, _ = im.shape
    im = im[h//2-112:h//2+112, w//2-112:w//2+112]
    
    rawim = np.copy(im).astype('uint8')
    
    # Shuffle axes to c01
    im = np.swapaxes(np.swapaxes(im, 1, 2), 0, 1)
    
    # Convert to BGR
    im = im[::-1, :, :]

    im = im - MEAN_VALUES
    return rawim, floatX(im[np.newaxis])

In [ ]:
progress = 0
def chunks(l, n):
    for i in xrange(0, len(l), n):
        end = i + n
        if end > len(l):
            end = len(l)
        yield l[i:end]

for chunk in chunks(dataset, 20):
    cnn_input = floatX(np.zeros((len(chunk), 3, 224, 224)))
    for i, image in enumerate(chunk):
        fn =  "allImages/" + image['path']
        try:
            im = plt.imread(fn)
            _,cnn_input[i] = prep_image(im)
        except IOError:
            continue
    features = get_cnn_features(cnn_input)
    for i, image in enumerate(chunk):
        image['cnn features'] = features[i]
    progress += 1
    print "Progress: ", progress * 1.0 * 100 / len(dataset)
    

In [ ]:
f = open('image_dataset_with_features_googlenet.pkl', 'wb')
cPickle.dump(dataset, f)
f.close()